## Probability

In [1]:
from __future__ import division
import numpy as np
from collections import Counter
import random
import scipy.stats

1\. Bobo the amoeba has a 25%, 25%, and 50% chance of descendants also have the same probabilities. What is the probability that Bobo’s lineage dies out?

```
p = 0.25 + 0.25*p + 0.5*(p^2)

solutions:
p = 0.5
p = 1 (not really sure what to think of this one)
```

We can write some code to simulate the scenario:

In [2]:
def generation(n=1): # n is number of living amoebas
    if n > 100:
        return True # consider survived
    else:
        next_gen = 0
        # for each amoeba in current generation count descendants
        for amoeba in xrange(n):
            # random number of descendants: 0-3
            descendants = np.random.randint(0,4)
            # 3 descendants are only two :)
            if descendants == 3:
                descendants = 2
            next_gen += descendants
        if next_gen == 0:
            return False # died out
        else:
            return generation(n=next_gen)

experiments = 10000
survived = 0
for experiment in xrange(experiments):
    if generation():
        survived += 1
print 'survival rate: {}% (in {} experiments)'.format(100*survived/experiments, experiments)

survival rate: 50.39% (in 10000 experiments)


2\. In any 15-minute interval, there is a 20% probability that you will see at least one shooting star. What is the probability that you see at least one shooting star in the period of an hour?

```
1 - p(0 stars)
1 - 0.8**4
0.5904
```

See whether the code agrees:

In [3]:
def hour():
    for quarter in xrange(4):
        if np.random.randint(0,5) == 0:
            return True
    return False

experiments = 10000
stars = 0
for experiment in xrange(experiments):
    if hour():
        stars += 1
print 'saw at least one star: {}% (in {} experiments)'.format(100*stars/experiments, experiments)

saw at least one star: 59.47% (in 10000 experiments)


3\. How can you generate a random number between 1 - 7 with only a die?

```
roll twice (if both rolls are six start over)
there are 35 possibilities (6*6-1 for ignoring two sixes), all equally likely
just divide them into 7 "buckets"
this will return numbers from 1 to 35 (x, y being first and second roll): 6(x-1) + y
we can then just do modulo 7 and add 1
```

See if it works (it is not ideal but I am going to blame numpy non-randomness):

In [4]:
x = np.random.randint(1, 7, size=10000)
y = np.random.randint(1, 7, size=10000)
print Counter((6*(x-1) + y) % 7 + 1)

Counter({2: 1679, 1: 1418, 3: 1418, 4: 1410, 6: 1386, 7: 1353, 5: 1336})


4\. How can you get a fair coin toss if someone hands you a coin that is weighted to come up heads more often than tails?

```
Flip twice:
- HT >> "Heads", done
- TH >> "Tails", done
- HH or TT >> start over

Sanity check: If coin is fair the result is the first toss which is fair.
It may take a while if the coin is not fair but eventualy there will be either HT or TH, both the same likely.
```

Lets do a simulation:

In [5]:
def flip(p):
    return 'H' if random.random() < p else 'T'

experiments = 10000
results = []
p = 0.8
for experiment in xrange(experiments):
    x = flip(p)
    y = flip(p)
    if x != y:
        results.append(x)

print Counter(results)

Counter({'T': 1647, 'H': 1601})


5\. You have an 50-50 mixture of two normal distributions with the same standard deviation. How far apart do the means need to be in order for this distribution to be bimodal?

```
Definition: Bimodal distribution is a continuous probability distribution with two different modes. These appear as distinct peaks (local maxima) in the probability density function.

The probability of x in the mixture is given by p(x)=0.5N(x;μ1,σ)+0.5N(x;μ2,σ) which is symmetric around (μ1+μ2)/2. The mixture is bimodal if and only if |μ1−μ2|>2σ.
```

6\. Given draws from a normal distribution with known parameters, how can you simulate draws from a uniform distribution?

```
There is actually a technique called Probability Integral Transform (aka Universality of the Uniform) which says "If you plug in a random variable into its own CDF, you get a Uniform distribution." So just plug your draw into the CDF of the normal distribution.
```

Not really a proof but this is the idea in code:

In [6]:
def draw():
    draw = scipy.stats.norm.rvs()
    return int(scipy.stats.norm.cdf(draw)*10) #multiply by 10 and round down to int

results = []
experiments = 10000
for experiment in xrange(experiments):
    results.append(draw())

print Counter(results)

Counter({8: 1013, 3: 1011, 1: 1007, 9: 1005, 7: 1003, 4: 1002, 0: 1000, 6: 1000, 2: 982, 5: 977})


7\. A certain couple tells you that they have two children, at least one of which is a girl. What is the probability that they have two girls?

```
Assuming the chances for boy vs girl are equal, the sample space is {BB, BG, GB, GG}.
We know their case is not BB, which leaves us with BG, GB, GG. The chance of two girls is 1/3.
```

Some experiment code:

In [7]:
x = np.random.randint(0, 2, size=10000) # 1st child (girl 0, boy 1)
y = np.random.randint(0, 2, size=10000) # 2nd child (girl 0, boy 1)

# two girls: x + y = 0
# at least one girl: x + y <=1
print sum(x + y == 0) / sum(x + y <= 1)

0.337175410705


8\. You have a group of couples that decide to have children until they have their first girl, after which they stop having children. What is the expected gender ratio of the children that are born? What is the expected number of children each couple will have?

```
Intuitively: All couples have their first child and those children are 50-50 boys and girls. Half of the couples (those with boys) has another child and those children are again 50-50 boys and girls. This continues but the ration remains and there is the same number of girls and boys. Expected number of children for each couple is 2.
```

In [8]:
def generation(n=10, g=0, b=0): # n is number of couples
    if n == 0:
        return g, b
    else:
        children = np.random.randint(0, 2, size=n) # girl 0, boy 1
        g += sum(children == 0)
        b += sum(children == 1)
        return generation(n=sum(children == 1), g=g, b=b)

couples = 10000
girls, boys = generation(n=couples)
print '{} girls, {} boys, {} children per couple'.format(girls, boys, (girls+boys)/couples)

10000 girls, 9879 boys, 1.9879 children per couple


9\. How many ways can you split 12 people into 3 teams of 4?

```
12 choose 4 for the first group = 495
8 choose 4 for the second group = 70
4 choose 4 for the third group = 1
multiply those numbers and divide by 3! because we do not care about the order of the groups

result: (495 * 70 * 1) / 3! = 5775
```